In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
TRAIN_DIR = "/kaggle/input/fruit-and-vegetable-image-recognition/train"
VALIDATION_DIR = "/kaggle/input/fruit-and-vegetable-image-recognition/validation"
TEST_DIR = "/kaggle/input/fruit-and-vegetable-image-recognition/test"

In [ ]:
# directories
TRAIN_DIR = "/content/gdrive/MyDrive/fruit-and-vegetable-image-recognition/train"
VALIDATION_DIR = "/content/gdrive/MyDrive/fruit-and-vegetable-image-recognition/validation"
TEST_DIR = "/content/gdrive/MyDrive/fruit-and-vegetable-image-recognition/test"

In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

In [ ]:
training_images = image_dataset_from_directory(
    TRAIN_DIR,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE
)

In [ ]:
validation_images = image_dataset_from_directory(
    VALIDATION_DIR,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE
)

In [ ]:
testing_images = image_dataset_from_directory(
    TEST_DIR,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE
)

In [ ]:
class_names = training_images.class_names
class_names

In [ ]:
def normalize(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

training_images = training_images.map(normalize)
validation_images =  validation_images.map(normalize)
testing_images =  testing_images.map(normalize)

In [ ]:
training_images = training_images.prefetch(tf.data.experimental.AUTOTUNE)
validation_images = validation_images.prefetch(tf.data.experimental.AUTOTUNE)
testing_images = testing_images.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
model = tf.keras.models.Sequential([
    # input layer
    tf.keras.Input(IMAGE_SIZE + (3,)),

    # preprocess layers
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomFlip('vertical'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),

    # CNN layers
    tf.keras.layers.Conv2D(32, kernel_size=(2,2), input_shape=[224,224,3], activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, kernel_size=(2,2), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, kernel_size=(2,2), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512, kernel_size=(2,2), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(1024, kernel_size=(2,2), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.BatchNormalization(),

    # max pooling layer
    tf.keras.layers.GlobalMaxPooling2D(),

    # output layer
    tf.keras.layers.Dense(36)
])

In [ ]:
def make_model(image_shape, filters, activation, initializer, bias):

    with tf.device('/gpu:0'):
        input_shape = image_shape + (3,)
        inputs = tf.keras.Input(shape=input_shape)
        aug = tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal')(inputs)
        aug = tf.keras.layers.experimental.preprocessing.RandomFlip('vertical')(aug)
        aug = tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)(aug)
        aug = tf.keras.layers.experimental.preprocessing.RandomZoom(0.1)(aug)
        x = tf.keras.layers.Conv2D(filters, kernel_size=(3), input_shape=[224,224,3], activation=activation, kernel_initializer=initializer, use_bias=bias)(aug)
        x = tf.keras.layers.MaxPool2D(2,2)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv2D(filters, kernel_size=(3), activation=activation, kernel_initializer=initializer, use_bias=bias)(x)
        x = tf.keras.layers.MaxPool2D(2,2)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv2D(filters, kernel_size=(3), activation=activation, kernel_initializer=initializer, use_bias=bias)(x)
        x = tf.keras.layers.MaxPool2D(2,2)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv2D(filters, kernel_size=(3), activation=activation, kernel_initializer=initializer, use_bias=bias)(x)
        x = tf.keras.layers.MaxPool2D()(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv2D(filters, kernel_size=(3), activation=activation, kernel_initializer=initializer, use_bias=bias)(x)
        x = tf.keras.layers.MaxPool2D(2,2)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.GlobalMaxPooling2D()(x)
        output = tf.keras.layers.Dense(36)(x)
        model = tf.keras.Model(inputs, output)

        loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        metrics=['accuracy']

        model.compile(loss=loss_function,
                      optimizer = optimizer,
                      metrics=metrics)

        return model

In [ ]:
def train_model(model, name):
    print(f'train {name}')
    pengurangan_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=10, verbose=1)
#     pengurangan_lr = reduce_lr_callback = LearningRateScheduler(scheduler)
    history = model.fit(training_images,
                        validation_data=validation_images,
                        epochs=100,
                        callbacks=[pengurangan_lr])

    model.save(f'weight_{name}.h5')
    np.save(f'history_{name}.npy',history.history)
    print()

In [ ]:
filters = 32
activation = 'relu'
bias = True
initializer = 'he_uniform'

for activation in ['relu','gelu', 'leakyrelu']:
    name = f'activations_{activation}'
    model = make_model(IMAGE_SIZE, filters, activation, initializer, bias)
    train_model(model, name)

In [ ]:
pred = model.predict(testing_images)

In [ ]:
labels_array = np.array([])
pred_array = np.array([])

for x, y in testing_images.take(1):
    pred_prob = model.predict(x)
    labels_array = np.concatenate([labels_array, y])
    pred_class = np.argmax(pred_prob, axis=1)
    pred_array = np.concatenate([pred_array, pred_class])  

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("Accuracy On Test Dataset: ", accuracy_score(labels_array, pred_array))

In [ ]:
model.save('capstone_v2.h5')

In [ ]:
model.save(f'weight_capstone2.h5')
np.save(f'history_1.npy',history.history)

In [ ]:
model.save('capstone_v2.keras')

In [ ]:
acc = [0.] + history.history['accuracy']
val_acc = [0.] + history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.5])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
import tensorflow as tf

# Load the model from the saved path
model = tf.keras.models.load_model("/kaggle/working/weight_capstone2.h5")

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
# Simpan model TFLite ke file
with open("model.tflite", "wb") as f:
    f.write(tflite_model)